In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import chiscrape
import util
import networkx as nx
import pandas as pd
import urllib
import matplotlib.pyplot as plt
%matplotlib inline
import re

In [73]:
path = "C:/Users/vanguiano/Desktop/CoCWebsite/chiscrape/data/run_on_2_6_1_14/"
data = pd.read_csv(path+'data_output_2_6_1_14.csv', encoding = 'iso-8859-1', index_col = 0)
unique_domains = pd.read_csv(path+'unique_domains_2_6_1_14.csv', encoding = 'iso-8859-1', index_col = 0)
dead_links = pd.read_csv(path+'dead_links_2_6_1_14.csv', index_col = 0)

In [4]:
data.sort_values(by='num_email_addresses', ascending=False)

,dept,title,url,button,i_want_to,nav,num_email_addresses,email_addresses,pdf_count,pdf_urls,ext_link_count,ext_links,outside_domain,description_words
1536,NaN,\r\n City of Chicago :: Aldermanic Offices\r\n,https://www.cityofchicago.org/city/en/depts/ba...,False,True,True,47,mailto:ward01@cityofchicago.org; mailto:ward02...,1,https://www.cityofchicago.org/content/dam/city...,30,http://aldermanmoreno.com/; http://www.dowellf...,chicago47.org; 50thwardchicago.com; 12thwardch...,aldermen; staff; provide; assistance; business...
3154,-es,\r\n City of Chicago :: Oficinas Municipale...,https://www.cityofchicago.org/city/en/depts/ba...,False,True,True,47,mailto:ward01@cityofchicago.org; mailto:ward02...,1,https://www.cityofchicago.org/content/dam/city...,30,http://aldermanmoreno.com/; http://www.dowellf...,chicago47.org; 50thwardchicago.com; 12thwardch...,los; concejales; su; personal; ofrecen; ayuda;...
2441,NaN,\r\n City of Chicago :: FOIA Contact Inform...,https://www.cityofchicago.org/city/en/narr/foi...,False,True,True,35,mailto:MOfoia@cityofchicago.org; mailto:311foi...,0,NaN,5,http://chicityclerk.com/office-info/foia/; htt...,chicagocitytreasurer.com; ilga.gov; illinoisat...,foia; requests; must; submitted; directly; app...
3693,_info,\r\n City of Chicago :: Technical Assistanc...,https://www.cityofchicago.org/city/en/depts/dc...,False,True,True,26,mailto:ghennings@pcc-chgo.org; mailto:ghenning...,0,NaN,0,NaN,NaN,technical; assistance-community; former; housi...
2536,_info,\r\n City of Chicago :: Foreclosure Prevent...,https://www.cityofchicago.org/city/en/depts/dc...,False,True,True,15,mailto:dwilliams@bethelnewlife.com; mailto:mwo...,1,https://www.cityofchicago.org/content/dam/city...,13,http://www.bethelnewlife.org; http://www.thech...,picbchicago.org; nwshc.org; caslservice.org; n...,foreclosure; prevention; housing; counseling; ...
5306,o_generated,\r\n City of Chicago :: Innovation and Tech...,https://www.cityofchicago.org/city/en/depts/do...,False,True,True,14,mailto:laticia.latham@cityofchicago.org; mailt...,0,NaN,0,NaN,NaN,danielle; dumerer; cio; commissioner; danielle...
2743,_info,\r\n City of Chicago :: Independent Living ...,https://www.cityofchicago.org/city/en/depts/dc...,False,True,True,13,mailto:jsgonsja@bync.org; mailto:Drennie@bicke...,0,NaN,0,NaN,NaN,small; accessible; repairs; seniors; sarfs; ag...
3568,NaN,\r\n City of Chicago :: Small Business Deve...,https://www.cityofchicago.org/city/en/depts/ba...,False,True,True,8,mailto:info@bethelnewlife.org; mailto:i-conda@...,0,NaN,12,http://www.illinois.gov/dceo/SmallBizAssistanc...,illinois.gov; worldbusinesschicago.com; amlega...,sbdcs; provide; businesses; free; professional...
1789,-es,\r\n City of Chicago :: Centros de Desarrol...,https://www.cityofchicago.org/city/en/depts/ba...,False,True,True,8,mailto:info@bethelnewlife.org; mailto:i-conda@...,0,NaN,12,http://www.illinois.gov/dceo/SmallBizAssistanc...,illinois.gov; worldbusinesschicago.com; amlega...,los; sbdc; ofrecen; los; negocios; guía; profe...
1887,_info,\r\n City of Chicago :: Chicago Film Office...,https://www.cityofchicago.org/city/en/depts/dc...,False,True,True,8,mailto:filmoffice@cityofchicago.org; mailto:kj...,0,NaN,13,http://www.transitchicago.com/film.aspx; http:...,millenniumpark.org; chicagoparkdistrict.com; t...,dcase; homepage; film; office; services/contac...


In [10]:
urls = list(data.url)
depts = []
for url in urls:
    depts.append((get_department(url),url))

In [11]:
def get_department(url):
    try:
        m = re.search('depts/(.+?)/', url).group(1)
    except:
        try:
            m = re.search('depts/(.+?)\.', url).group(1)
        except:
            return 'None'

    return m

In [12]:
check = pd.DataFrame(depts, columns = ['dept', 'url'], index=list(range(1,len(data)+1)))

In [13]:
data['dept'] = check.dept

In [14]:
data['services'] = data.url.str.contains('svcs')

In [19]:
data['title'] = data.title.fillna('None')
cleaned = [title.strip() for title in list(data.title)]

In [20]:
data['title'] = cleaned

In [21]:
dept_ref = pd.read_csv('dept_reference.csv')

In [22]:
dept_ref_dict = {}
for index in dept_ref.index:
    dept_ref_dict[dept_ref.iloc[index][0]] = dept_ref.iloc[index][1]

In [23]:
data['dept_name'] = data.dept.apply(lambda x: dept_ref_dict.get(x,'Other'.strip()))

In [24]:
digestable_data = data[['dept_name', 'title', 'url', 'services','button', 'i_want_to', 'nav',
       'num_email_addresses', 'email_addresses', 'pdf_count',
       'ext_link_count', 'outside_domain']]

In [ ]:
data.to_csv('data_output_2_6_1_14_cleaned.csv')

In [61]:
pdf_counts = data.groupby('dept_name')['pdf_count'].agg(['count','sum'])

In [62]:
pdf_counts = pdf_counts.sort_values('sum',ascending=False)
pdf_counts.columns = [['Number of Web Pages', 'Total PDFs']]

In [63]:
pdf_counts = pdf_counts.rename_axis('Department Name')
pdf_counts = pdf_counts.reset_index()

In [28]:
pdf_counts = pdf_counts[pdf_counts['Department Name'] != 'None']
pdf_counts = pdf_counts[pdf_counts['Department Name'] != 'Other']

In [65]:
pdf_counts.to_csv('departments_pdf_counts.csv', index=False)

In [60]:
pdf_counts

,Department Name,Number of Web Pages,Total PDFs
0,Department of Procurement Services,299,6852
1,Department of Planning and Development,546,6619
2,Department of Water Management,149,2596
3,Board of Ethics,137,2428
4,Department of Human Resources,126,2365
5,Department of Business Affairs and Consumer Pr...,294,1753
6,Department of Public Health,288,1698
8,Department of Finance,311,913
9,Office of Budget and Management,70,478
10,Commission on Human Relations,59,433


In [31]:
data[data.dept == 'dps'].sort_values('pdf_count', ascending = False)

,dept,title,url,button,i_want_to,nav,num_email_addresses,email_addresses,pdf_count,pdf_urls,ext_link_count,ext_links,outside_domain,description_words,services,dept_name
29,dps,City of Chicago :: Current Bid Opportunities,https://www.cityofchicago.org/city/en/depts/dp...,False,True,True,0,NaN,133,https://www.cityofchicago.org/content/dam/city...,2,https://webapps1.cityofchicago.org/EDSWeb/appm...,webapps1.cityofchicago.org; author.cityofchica...,eprocurement; bid; opportunities; solicitation...,True,Department of Procurement Services
1245,dps,City of Chicago :: 2011 Job Order Contracts (JOC),https://www.cityofchicago.org/city/en/depts/dp...,False,True,True,0,NaN,125,https://www.cityofchicago.org/content/dam/city...,0,NaN,NaN,january; contract; po; release; general; const...,False,Department of Procurement Services
1267,dps,City of Chicago :: 2013 PreBid and Pre-Submitt...,https://www.cityofchicago.org/city/en/depts/dp...,False,True,True,0,NaN,122,https://www.cityofchicago.org/dam/city/depts/d...,0,NaN,NaN,january; prebid; conference; attendees; foster...,False,Department of Procurement Services
1258,dps,City of Chicago :: 2012 Pre-Bid and Pre-Submit...,https://www.cityofchicago.org/city/en/depts/dp...,False,True,True,0,NaN,116,https://www.cityofchicago.org/content/dam/city...,0,NaN,NaN,january; prebid; conference; attendees; specs;...,False,Department of Procurement Services
1249,dps,City of Chicago :: 2011 PreBid Conference Atte...,https://www.cityofchicago.org/city/en/depts/dp...,False,True,True,0,NaN,113,https://www.cityofchicago.org/content/dam/city...,0,NaN,NaN,january; prebid; conference; attendees; remote...,False,Department of Procurement Services
1177,dps,City of Chicago :: 2007 Pre-Bid Conference Att...,https://www.cityofchicago.org/city/en/depts/dp...,False,True,True,0,NaN,108,https://www.cityofchicago.org/content/dam/city...,0,NaN,NaN,january; prebid; conference; attendees; interi...,False,Department of Procurement Services
1288,dps,City of Chicago :: 2015 PreBid and PreSubmitta...,https://www.cityofchicago.org/city/en/depts/dp...,False,True,True,0,NaN,103,https://www.cityofchicago.org/content/dam/city...,0,NaN,NaN,january; prebid; conference; attendees; street...,False,Department of Procurement Services
1301,dps,City of Chicago :: 2016 PreBid and PreSubmitta...,https://www.cityofchicago.org/city/en/depts/dp...,False,True,True,0,NaN,103,https://www.cityofchicago.org/content/dam/city...,0,NaN,NaN,january; prebid; conference; attendees; compre...,False,Department of Procurement Services
1318,dps,City of Chicago :: 2017 PreBid and PreSubmitta...,https://www.cityofchicago.org/city/en/depts/dp...,False,True,True,0,NaN,102,https://www.cityofchicago.org/content/dam/city...,0,NaN,NaN,january; prebid; conference; attendees; runway...,False,Department of Procurement Services
1276,dps,City of Chicago :: 2014 PreBid and Pre-Submitt...,https://www.cityofchicago.org/city/en/depts/dp...,False,True,True,0,NaN,94,https://www.cityofchicago.org/content/dam/city...,0,NaN,NaN,january; prebid; conference; attendees; south;...,False,Department of Procurement Services


In [32]:
data.pdf_urls = data.pdf_urls.fillna('None')
pdf_links_list = []
for index in data.index[data.pdf_urls != 'None']:
    pdf_links_list.append((data.dept_name[index],data.url[index],data.pdf_urls[index]))
    
    

In [33]:
full_pdf_links_list = []
for page in pdf_links_list:
    for url in page[2].split(';'):
        full_pdf_links_list.append((page[0],page[1],url))

In [37]:
all_pdfs = pd.DataFrame(full_pdf_links_list, columns = ['Department', 'From URL', 'PDF Url'])

30542

In [58]:
all_pdfs.to_csv('all_Chicago_pdfs.csv',index=False)

In [42]:
all_pdfs_nodup = all_pdfs.drop_duplicates(subset='PDF Url')

In [43]:
all_pdfs_nodup.to_csv('all_Chicago_pdfs_nodups.csv', index = False)

In [57]:
count_by_dept = all_pdfs_nodup.groupby('Department').size().sort_values(ascending = False)
count_by_dept.to_csv('pdf_count_by_dept.csv')
count_by_dept


Department
Department of Procurement Services                        6733
Department of Planning and Development                    6409
Department of Water Management                            2545
Department of Human Resources                             2352
Board of Ethics                                           2180
Department of Business Affairs and Consumer Protection    1535
None                                                      1470
Department of Public Health                               1066
Department of Finance                                      821
Office of Budget and Management                            408
Commission on Human Relations                              361
Department of Buildings                                    310
Department of Innovation and Technology                    307
Chicago Police Board                                       263
Department of Transportation                               215
License Appeal Commission                   

In [53]:
all_pdfs_nodup.shape

(28248, 3)

In [54]:
all_pdfs.shape

(30542, 3)

In [70]:
all_pdfs[all_pdfs.duplicated(subset='PDF Url')].to_csv('duplicates.csv')

In [361]:
data.pdf_urls[data.dept_name == 'Department of Finance'][255]

'https://www.cityofchicago.org/content/dam/city/depts/rev/supp_info/TaxApplicationandAffidavitForms/Amusement_Tax_Exemption_Application_Finance.pdf; https://www.cityofchicago.org/content/dam/city/depts/rev/supp_info/TaxApplicationandAffidavitForms/DJApplication_Finance.pdf; https://www.cityofchicago.org/content/dam/city/depts/rev/supp_info/TaxRulingsandRegulations/Amusement_Ruling_4.pdf; https://www.cityofchicago.org/content/dam/city/depts/rev/supp_info/TaxApplicationandAffidavitForms/SmallNewBusExemptionApplication.pdf'

In [313]:
data[data.dept.str.contains('dps')].sort_values('pdf_count', ascending=False).url.iloc[29]

'https://www.cityofchicago.org/city/en/depts/dps/supp_info/2002_april_bid_tabulations.html'

In [ ]:
unique_city_of_chicago = list(unique_domains[unique_domains.domain_url.str.contains('.cityofchicago.')].domain_url.unique())
unique_city_of_chicago.remove('dev.cityofchicago.org.s3-website-us-east-1.amazonaws.com')
unique_city_of_chicago.sort()
unique_city_of_chicago


In [25]:
unique_city_of_chicago = unique_city_of_chicago[:-1]
len(unique_city_of_chicago)
edges = []
for url in unique_city_of_chicago:
    edges.append(('cityofchicago.org',url))
edges

In [136]:
G = nx.Graph()
for edge in edges:
    G.add_edge(*edge)
pos=nx.get_node_attributes(G,'pos')
nx.draw_networkx(G, node_color = 'gray', edge_color = 'black', width = 0.1, node_size = 20, with_labels = True, font_size=25)
#nx.draw_networkx_labels(G, label_list, fontsize = .5)
plt.rcParams["figure.figsize"] = [40,40]
plt.title("City of Chicago Subdomains, as of February 2018", fontsize = 20)
plt.savefig("cityofchicago_urls.png")

NameError: name 'G' is not defined

In [ ]:
edges = []
for edge0 in range(len(edge_0)):
    for edge1 in edge_1[edge0]:
        edges.append((edge_0[edge0], edge1.strip()))
    

In [ ]:
edge_1 = []
for x in data.outside_domain:
    try:
        edge_1.append(x.split(';'))
        
    except:
        edge_1.append([])

In [ ]:
def url_netloc(url):
    parsed_url = urllib.parse.urlparse(url)
    netloc = parsed_url.netloc
    return netloc

In [ ]:
from_domains = set()
for url in data.url:
    parsed_url = urllib.parse.urlparse(url)
    netloc = parsed_url.netloc
    from_domains.add(netloc)

In [ ]:
unique_domains = set()
for x in data.outside_domain:
    try:
        domains = x.split(';')
        for domain in domains:
            stripped = domain.strip()
            unique_domains.add(stripped)
    except:
        pass